In [36]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import os

In [2]:
gr_dir = 'GOODREADS/'
nyt_dir = 'NYT/'

In [76]:
def normalize_name(name):
    if pd.isna(name):
        return ''
    name = str(name)
    name = name.lower()
    name = ' '.join(name.split())
    name = name.replace('.', '')
    name = ''.join(c for c in name if c.isalnum() or c.isspace())
    return name.strip()

In [77]:
import pandas as pd
from rapidfuzz import process, fuzz

def fuzzy_merge(df1, df2, key1, key2, threshold=90):

    choices = df2[key2].dropna().astype(str).unique()
    choices_list = choices.tolist()

    # Tìm best match cho từng giá trị trong df1
    def find_best_match(name):
        if pd.isna(name):
            return None, 0
        name = str(name)
        match = process.extractOne(name, choices_list, scorer=fuzz.ratio)
        if match and match[1] >= threshold:
            return match[0]  # trả về tên khớp trong df2
        return None

    # Ánh xạ tên trong df1 → tên khớp tốt nhất trong df2 (nếu có)
    df1 = df1.copy()
    df1['_matched_key'] = df1[key1].apply(lambda x: find_best_match(x))

    # Chỉ giữ những thằng có match
    df1_matched = df1.dropna(subset=['_matched_key'])

    # Merge với df2 (inner join thực sự)
    result = df1_matched.merge(
        df2,
        left_on='_matched_key',
        right_on=key2,
        how='inner',
        suffixes=('_left', '_right')
    )

    # Dọn dẹp
    result = result.drop(columns=['_matched_key'])
    return result

## Goodreads

**Ref:**
https://cseweb.ucsd.edu/~jmcauley/datasets/goodreads.html#datasets

### goodreads_books

In [3]:
goodreads_books_cols = [
    'book_id',                # ID chính để gắn reviews sau này
    'title',                  # để merge
    'author',                # list dạng string, cần xử lý
    'publication_year',       # năm xuất bản (kiểm tra chéo)
    'average_rating',         # rating Goodreads (rất quan trọng)
    'ratings_count',          # số lượt rating Goodreads
    'description',            # để extract topic nếu cần
    'isbn',                   # dùng merge với Amazon nếu có
    'isbn13'
]
gr_books_df = pd.read_parquet(gr_dir + 'goodreads_books.parquet', 
                              engine='fastparquet', columns=goodreads_books_cols)

In [29]:
print(gr_books_df.columns)
gr_books_df.head()


Index(['book_id', 'title', 'author', 'publication_year', 'average_rating',
       'ratings_count', 'description', 'isbn', 'isbn13'],
      dtype='object')


,book_id,title,author,publication_year,average_rating,ratings_count,description,isbn,isbn13
0,5333265,W.C. Fields: A Life on Film,Ronald J. Fields,1984,4.00,3,,0312853122,9780312853129
1,1333909,Good Harbor,Anita Diamant,2001,3.23,10,"Anita Diamant's international bestseller ""The ...",0743509986,9780743509985
2,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",Barbara Hambly,1987,4.03,140,Omnibus book club edition containing the Ladie...,,
3,6066819,Best Friends Forever,Jennifer Weiner,2009,3.49,51184,Addie Downs and Valerie Adler were eight when ...,0743294297,9780743294294
4,287140,Runic Astrology: Starcraft and Timekeeping in ...,Nigel Pennick,<NA>,3.40,15,,0850308712,9780850308716


In [ ]:
print(gr_books_df.isna().sum(),'\n')
print(gr_books_df.shape,'\n')

book_id                  0
title                    0
author                   0
publication_year    599120
average_rating           0
ratings_count            0
description              0
isbn                     0
isbn13                   0
dtype: int64 

(2360118, 9) 



In [5]:
gr_books_df.dropna(axis=0,how='any',inplace=True)

In [27]:
gr_books_df.dtypes

book_id              object
title                object
author               object
publication_year      Int64
average_rating      float64
ratings_count         int64
description          object
isbn                 object
isbn13               object
dtype: object

In [26]:
gr_books_df['publication_year'] = pd.to_numeric(gr_books_df['publication_year'], errors='coerce').astype('Int64')
gr_books_df['average_rating'] = pd.to_numeric(gr_books_df['average_rating'], errors='coerce').astype('float64')
gr_books_df['ratings_count'] = pd.to_numeric(gr_books_df['ratings_count'], errors='coerce').astype('int64')

### goodreads_reviews

In [6]:
goodreads_reviews_cols = [
    'book_id',                # key để group theo sách
    'rating',                 # 1–5 sao
    'review_text',            # <<<< TEXT chính để extract public/expert
    'n_votes',                # <<<< rất quan trọng để tách expert review
    'date_added',             # có thể dùng để lấy early reviews
    'date_updated'
]
gr_reviews_df = pd.read_parquet(gr_dir + 'goodreads_reviews_dedup.parquet',
                                 engine='fastparquet',
                                 columns=goodreads_reviews_cols)

In [7]:
print(gr_reviews_df.columns)
gr_reviews_df.head()

Index(['book_id', 'rating', 'review_text', 'n_votes', 'date_added',
       'date_updated'],
      dtype='object')


,book_id,rating,review_text,n_votes,date_added,date_updated
0,24375664,5,Mind blowingly cool. Best science fiction I've...,16,Fri Aug 25 13:55:02 -0700 2017,Mon Oct 09 08:55:59 -0700 2017
1,18245960,5,This is a special book. It started slow for ab...,28,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017
2,6392944,3,I haven't read a fun mystery book in a while a...,6,Mon Jul 24 02:48:17 -0700 2017,Sun Jul 30 09:28:03 -0700 2017
3,22078596,4,"Fun, fast paced, and disturbing tale of murder...",22,Mon Jul 24 02:33:09 -0700 2017,Sun Jul 30 10:23:54 -0700 2017
4,6644782,4,A fun book that gives you a sense of living in...,8,Mon Jul 24 02:28:14 -0700 2017,Thu Aug 24 00:07:20 -0700 2017


In [47]:
gr_reviews_df.isna().sum()

book_id         0
rating          0
review_text     0
n_votes         0
date_added      0
date_updated    0
dtype: int64

### merge goodread

In [32]:
gr = gr_books_df.merge(gr_reviews_df,on='book_id',how='left')

In [43]:
gr.head()

,book_id,title,author,publication_year,average_rating,ratings_count,description,isbn,isbn13,rating,review_text,n_votes,date_added,date_updated
0,5333265,W.C. Fields: A Life on Film,Ronald J. Fields,1984,4.00,3,,0312853122,9780312853129,4.0,Loved this book about WC Fields written by his...,1.0,Sun Jun 19 12:09:45 -0700 2016,Sat Jun 25 09:03:13 -0700 2016
1,1333909,Good Harbor,Anita Diamant,2001,3.23,10,"Anita Diamant's international bestseller ""The ...",0743509986,9780743509985,2.0,Not much I can say that many reviewers haven't...,0.0,Wed Apr 04 07:47:34 -0700 2012,Fri Apr 13 15:54:06 -0700 2012
2,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",Barbara Hambly,1987,4.03,140,Omnibus book club edition containing the Ladie...,,,3.0,Competent. I'm sure I read the first half year...,1.0,Fri Dec 18 14:06:12 -0800 2009,Wed Jan 20 05:04:01 -0800 2010
3,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",Barbara Hambly,1987,4.03,140,Omnibus book club edition containing the Ladie...,,,4.0,"While these books are excellent overall, I thi...",0.0,Tue Apr 04 07:18:36 -0700 2017,Sun May 07 17:11:44 -0700 2017
4,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",Barbara Hambly,1987,4.03,140,Omnibus book club edition containing the Ladie...,,,3.0,A collection of two novels about the mercenary...,0.0,Fri Oct 11 10:08:07 -0700 2013,Fri Oct 25 09:31:27 -0700 2013


In [ ]:
gr.isna().sum()

book_id                   0
title                     0
author                    0
publication_year    2703801
average_rating            0
ratings_count             0
description               0
isbn                      0
isbn13                    0
rating               280209
review_text          280209
n_votes              280209
date_added           280209
date_updated         280209
dtype: int64

In [48]:
gr.dropna(subset=['rating','review_text','n_votes','date_added','date_updated'],axis=0, inplace=True)

In [49]:
gr.isna().sum()

book_id                   0
title                     0
author                    0
publication_year    2605109
average_rating            0
ratings_count             0
description               0
isbn                      0
isbn13                    0
rating                    0
review_text               0
n_votes                   0
date_added                0
date_updated              0
dtype: int64

## Books sales and ratings

**Ref:** https://www.kaggle.com/datasets/thedevastator/books-sales-and-ratings

### Sale_data

In [39]:
books_sale_cols = [
    'Book Name',              # → title
    'Author',                 # → author
    'Book_average_rating',    # rating trung bình trên nền tảng này
    'Book_ratings_count',     # số lượt rating
    'genre',                  # thể loại (text)
    'gross sales',            # doanh thu tổng
    'publisher revenue',      # doanh thu nhà xuất bản
    'sale price',             # giá bán lẻ
    'sales rank',             # xếp hạng bán hàng (thấp = tốt)
    'units sold',             # <<<< GROUND-TRUTH CHÍNH <<<<
    'Author_Rating',          # None / Novice / Intermediate / Famous → critical success mạnh
    'Publishing Year'         # năm xuất bản
]
sale_df = pd.read_csv('Books_Sale.csv',usecols=books_sale_cols)

In [38]:
sale_df.head()

,Publishing_Year,Book_Name,Author,Author_Rating,Book_Average_Rating,Book_Ratings_Count,Genre,Gross_Sales,Publisher_Revenue,Sale_Price,Sales_Rank,Units_Sold
0,1975.0,Beowulf,"Unknown, Seamus Heaney",Novice,3.42,155903,genre fiction,34160.0,20496.0,4.88,1,7000
1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",Intermediate,4.23,145267,genre fiction,12437.5,7462.5,1.99,2,6250
2,2015.0,Go Set a Watchman,Harper Lee,Novice,3.31,138669,genre fiction,47795.0,28677.0,8.69,3,5500
3,2008.0,When You Are Engulfed in Flames,David Sedaris,Intermediate,4.04,150898,fiction,41250.0,24750.0,7.50,3,5500
4,2011.0,Daughter of Smoke & Bone,Laini Taylor,Intermediate,4.04,198283,genre fiction,37952.5,22771.5,7.99,4,4750


In [40]:
sale_cols = sale_df.columns
s_new_cols = []
for col in sale_cols:
    proper_col = col.title()
    proper_col = proper_col.replace(' ', '_')
    s_new_cols.append(proper_col)

sale_df.columns = s_new_cols

In [75]:
sale_df.head()

,Publishing_Year,Book_Name,Author,Author_Rating,Book_Average_Rating,Book_Ratings_Count,Genre,Gross_Sales,Publisher_Revenue,Sale_Price,Sales_Rank,Units_Sold
0,1975.0,Beowulf,"Unknown, Seamus Heaney",Novice,3.42,155903,genre fiction,34160.0,20496.0,4.88,1,7000
1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",Intermediate,4.23,145267,genre fiction,12437.5,7462.5,1.99,2,6250
2,2015.0,Go Set A Watchman,Harper Lee,Novice,3.31,138669,genre fiction,47795.0,28677.0,8.69,3,5500
3,2008.0,When You Are Engulfed In Flames,David Sedaris,Intermediate,4.04,150898,fiction,41250.0,24750.0,7.50,3,5500
4,2011.0,Daughter Of Smoke & Bone,Laini Taylor,Intermediate,4.04,198283,genre fiction,37952.5,22771.5,7.99,4,4750


In [74]:
sale_df['Book_Name'] = sale_df['Book_Name'].str.title()

## New York Times BestSellers

**Ref:** https://www.kaggle.com/datasets/sujaykapadnis/new-york-times-bestsellers

### nyt_titles

In [10]:
nyt_titles_cols = [
    'id',              # ID nội bộ NYT (không dùng để merge)
    'title',           # <<< DÙNG ĐỂ MERGE (chính)
    'author',          # <<< DÙNG ĐỂ MERGE (chính)
    'year',            # năm debut
    'total_weeks',     # <<< SIÊU MẠNH: số tuần nằm trong list = longevity
    'first_week',      # tuần đầu tiên lên list (có thể dùng tính tốc độ lên hạng)
    'debut_rank',      # hạng tuần đầu tiên (1 = debut #1 → cực kỳ hiếm & mạnh)
    'best_rank'        # hạng cao nhất từng đạt (1 = từng #1)
]
nyt_titles_df = pd.read_csv(nyt_dir + 'nyt_titles.tsv',sep='\t',usecols=nyt_titles_cols)

In [51]:
nyt_titles_df.head()

,id,title,author,year,total_weeks,first_week,debut_rank,best_rank
0,0,"""H"" Is For Homicide",Sue Grafton,1991,15,1991-05-05,1,2
1,1,"""I"" Is For Innocent",Sue Grafton,1992,11,1992-04-26,14,2
2,10,''G'' Is For Gumshoe,Sue Grafton,1990,6,1990-05-06,4,8
3,100,A Dog'S Journey,W. Bruce Cameron,2012,1,2012-05-27,3,14
4,1000,Changing Faces,Kimberla Lawson Roby,2006,1,2006-02-19,11,14


In [ ]:
nyt_titles_df['title']=nyt_titles_df['title'].str.title()

### nyt_full

In [52]:
nyt_full_cols = [
    'title',   # để nối với nyt_titles.tsv
    'year',
    'week',
    'rank'        # hạng tuần đó (1–10 hoặc 1–15 tùy năm)
]

nyt_full_df = pd.read_csv(nyt_dir + 'nyt_full.tsv',sep='\t',usecols=nyt_full_cols)

In [53]:
nyt_full_df.head()

,year,week,rank,title
0,1931,1931-10-12,1,THE TEN COMMANDMENTS
1,1931,1931-10-12,2,FINCHE'S FORTUNE
2,1931,1931-10-12,3,THE GOOD EARTH
3,1931,1931-10-12,4,SHADOWS ON THE ROCK
4,1931,1931-10-12,5,SCARMOUCHE THE KING MAKER


In [54]:
nyt_full_df['title'] = nyt_full_df['title'].str.title()

In [58]:
nyt_full_df.head()

,year,week,rank,title
0,1931,1931-10-12,1,The Ten Commandments
1,1931,1931-10-12,2,Finche'S Fortune
2,1931,1931-10-12,3,The Good Earth
3,1931,1931-10-12,4,Shadows On The Rock
4,1931,1931-10-12,5,Scarmouche The King Maker


In [66]:
nyt_df = nyt_titles_df.merge(nyt_full_df, on='title',how='left')

In [67]:
nyt_df.head()

,id,title,author,year_x,total_weeks,first_week,debut_rank,best_rank,year_y,week,rank
0,0,"""H"" Is For Homicide",Sue Grafton,1991,15,1991-05-05,1,2,1991,1991-05-05,15
1,0,"""H"" Is For Homicide",Sue Grafton,1991,15,1991-05-05,1,2,1991,1991-05-12,5
2,0,"""H"" Is For Homicide",Sue Grafton,1991,15,1991-05-05,1,2,1991,1991-05-19,2
3,0,"""H"" Is For Homicide",Sue Grafton,1991,15,1991-05-05,1,2,1991,1991-05-26,2
4,0,"""H"" Is For Homicide",Sue Grafton,1991,15,1991-05-05,1,2,1991,1991-06-02,5


In [73]:
nyt_df = nyt_df.drop(columns='year_y')
nyt_df.rename(columns={'year_x':'year'})

,id,title,author,year,total_weeks,first_week,debut_rank,best_rank,week,rank
0,0,"""H"" Is For Homicide",Sue Grafton,1991,15,1991-05-05,1,2,1991-05-05,15
1,0,"""H"" Is For Homicide",Sue Grafton,1991,15,1991-05-05,1,2,1991-05-12,5
2,0,"""H"" Is For Homicide",Sue Grafton,1991,15,1991-05-05,1,2,1991-05-19,2
3,0,"""H"" Is For Homicide",Sue Grafton,1991,15,1991-05-05,1,2,1991-05-26,2
4,0,"""H"" Is For Homicide",Sue Grafton,1991,15,1991-05-05,1,2,1991-06-02,5
...,...,...,...,...,...,...,...,...,...,...
64249,999,Changes,Jim Butcher,2010,4,2010-04-25,3,1,1984-03-18,14
64250,999,Changes,Jim Butcher,2010,4,2010-04-25,3,1,2010-04-25,1
64251,999,Changes,Jim Butcher,2010,4,2010-04-25,3,1,2010-05-02,3
64252,999,Changes,Jim Butcher,2010,4,2010-04-25,3,1,2010-05-09,10


In [79]:
df = fuzzy_merge(nyt_df,sale_df,'title','Book_Name',threshold=85)
df.head()

,id,title,author,year_x,total_weeks,first_week,debut_rank,best_rank,week,rank,...,Author,Author_Rating,Book_Average_Rating,Book_Ratings_Count,Genre,Gross_Sales,Publisher_Revenue,Sale_Price,Sales_Rank,Units_Sold
0,1017,Chasing Harry Winston,Lauren Weisberger,2008,11,2008-06-15,5,4,2008-06-15,6,...,Lauren Weisberger,Novice,3.32,63342,genre fiction,558.88,335.328,4.99,1042,4440
1,1017,Chasing Harry Winston,Lauren Weisberger,2008,11,2008-06-15,5,4,2008-06-22,6,...,Lauren Weisberger,Novice,3.32,63342,genre fiction,558.88,335.328,4.99,1042,4440
2,1017,Chasing Harry Winston,Lauren Weisberger,2008,11,2008-06-15,5,4,2008-06-29,6,...,Lauren Weisberger,Novice,3.32,63342,genre fiction,558.88,335.328,4.99,1042,4440
3,1017,Chasing Harry Winston,Lauren Weisberger,2008,11,2008-06-15,5,4,2008-07-06,4,...,Lauren Weisberger,Novice,3.32,63342,genre fiction,558.88,335.328,4.99,1042,4440
4,1017,Chasing Harry Winston,Lauren Weisberger,2008,11,2008-06-15,5,4,2008-07-13,8,...,Lauren Weisberger,Novice,3.32,63342,genre fiction,558.88,335.328,4.99,1042,4440


In [82]:
print(df.shape,'\n')
print(df.columns,'\n')


(5211, 22) 

Index(['id', 'title', 'author', 'year_x', 'total_weeks', 'first_week',
       'debut_rank', 'best_rank', 'week', 'rank', 'Publishing_Year',
       'Book_Name', 'Author', 'Author_Rating', 'Book_Average_Rating',
       'Book_Ratings_Count', 'Genre', 'Gross_Sales', 'Publisher_Revenue',
       'Sale_Price', 'Sales_Rank', 'Units_Sold'],
      dtype='object') 



In [85]:
df = fuzzy_merge(df,gr,'title','title',threshold=85)

In [ ]:
columns_to_drop = [
    'title_left', 'Book_Name',     
    'author_left', 'Author',        
    'year_x', 'Publishing_Year',    
    'Book_Average_Rating',           
    'Book_Ratings_Count'    
]
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
df.rename(columns={
    'title_right': 'Title',
    'author_right': 'Author'
}, inplace=True)        

In [14]:
df['is_expert'] = df.groupby('Genre')['n_votes']\
                    .transform(lambda x: x >= x.quantile(0.90))\
                    .astype(int)

In [23]:
from sklearn.preprocessing import OrdinalEncoder

ord = df['Author_Rating'].unique().tolist()
ord_enc = OrdinalEncoder(categories=[ord])

df['Author_Rating_Encoded'] = ord_enc.fit_transform(df[['Author_Rating']])

In [34]:
desired_order = [
    #Overall book metadata
    'book_id',            
    'title',              
    'author',             
    'publication_year',   
    'description',
    'isbn', 'isbn13',
    'Genre',              # từ Sales (chuẩn thể loại)

    # New York Times critical success metrics
    'Author_Rating',      # Novice/Intermediate/Famous → critical proxy
    'Author_Rating_Encoded',
    'id',                 # ID NYT (giữ để trace)
    'total_weeks',        # NYT longevity – critical acclaim mạnh
    'best_rank',          # NYT peak
    'debut_rank',
    'first_week',

    # REVIEWS 
    'rating',             # rating của từng review (1–5)
    'review_text',        # text để extract sentiment
    'n_votes',            # <<< quan trọng nhất để tách expert/public
    'is_expert',          # cờ expert review (1 = expert)
    'date_added',   
    'date_updated',
    'average_rating',     # average rating của sách trên GR (public perception)
    'ratings_count',      # tổng số rating (popular success proxy)

    #sales data (ground-truth)
    'Units_Sold',       
    'Gross_Sales',
    'Publisher_Revenue',
    'Sale_Price',
    'Sales_Rank',
]
available_cols = [col for col in desired_order if col in df.columns]
df = df[available_cols]

In [35]:
print(df.shape,'\n')
print(df.columns,'\n')

(2527823, 26) 

Index(['book_id', 'publication_year', 'description', 'isbn', 'isbn13', 'Genre',
       'Author_Rating', 'Author_Rating_Encoded', 'id', 'total_weeks',
       'best_rank', 'debut_rank', 'first_week', 'rating', 'review_text',
       'n_votes', 'is_expert', 'date_added', 'date_updated', 'average_rating',
       'ratings_count', 'Units_Sold', 'Gross_Sales', 'Publisher_Revenue',
       'Sale_Price', 'Sales_Rank'],
      dtype='object') 



In [29]:
df.head()

,book_id,publication_year,description,isbn,isbn13,Genre,Author_Rating,id,total_weeks,best_rank,...,date_updated,average_rating,ratings_count,is_expert,Units_Sold,Gross_Sales,Publisher_Revenue,Sale_Price,Sales_Rank,Author_Rating_Encoded
0,426776,2008.0,"Meet Emmy, Leigh, and Adriana. Best friends si...",074356829X,9780743568296.0,genre fiction,Novice,1017,11,4,...,Sat Oct 11 17:47:13 -0700 2008,3.32,56,0,4440,558.88,335.328,4.99,1042,0.0
1,426776,2008.0,"Meet Emmy, Leigh, and Adriana. Best friends si...",074356829X,9780743568296.0,genre fiction,Novice,1017,11,4,...,Tue Feb 09 21:38:33 -0800 2010,3.32,56,0,4440,558.88,335.328,4.99,1042,0.0
2,426776,2008.0,"Meet Emmy, Leigh, and Adriana. Best friends si...",074356829X,9780743568296.0,genre fiction,Novice,1017,11,4,...,Tue Jul 21 23:35:38 -0700 2009,3.32,56,0,4440,558.88,335.328,4.99,1042,0.0
3,426776,2008.0,"Meet Emmy, Leigh, and Adriana. Best friends si...",074356829X,9780743568296.0,genre fiction,Novice,1017,11,4,...,Wed Apr 15 15:12:10 -0700 2009,3.32,56,0,4440,558.88,335.328,4.99,1042,0.0
4,426776,2008.0,"Meet Emmy, Leigh, and Adriana. Best friends si...",074356829X,9780743568296.0,genre fiction,Novice,1017,11,4,...,Mon Jun 08 20:42:27 -0700 2009,3.32,56,0,4440,558.88,335.328,4.99,1042,0.0


In [37]:
os.makedirs('DATA\SILVER', exist_ok=True)
df.to_csv('DATA\SILVER\gr_sale_nyt.csv', index = False)

KeyboardInterrupt: 